In [76]:
import emoji


In [77]:
#emoji dictionary
emoji_dict={
            "0":":\u2764\uFE0F:",
            "1":":baseball:",
            "2":":grinning_face_with_big_eyes:",
            "3":":disappointed_face:",
            "4":":fork_and_knife:"
             }


## LOading DATa

In [78]:
import pandas as pd
import numpy as np

In [79]:
train=pd.read_csv("train_emoji.csv",header=None)
test=pd.read_csv("test_emoji.csv",header=None)

In [80]:
x_train=train[0]
y_train=train[1]

x_test=test[0]
y_test=test[1]

In [81]:
for i in range(5):
    print(x_train[i],emoji.emojize(emoji_dict[str(y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much :❤️:
food is life 🍴


## loading Embedding Data

In [82]:
f=open("glove.6B.50d.txt",encoding="UTF-8")

In [83]:
embeddings={}  #diction of embedded data words of around 6 billion words


for i in f:
    values=i.split()
    word=values[0]
    coeff=np.asarray(values[1:],dtype="float")
    #print(word,coeff)
    embeddings[word]=coeff
    
f.close()

In [84]:
def embeddings_outputs(X):
    max_length=10
    embeddings_out=np.zeros((X.shape[0],max_length,50))
    
    for i in range(X.shape[0]):
        X[i]=X[i].split()
        
        for j in range(len(X[i])):
            try:
                embeddings_out[i][j]=embeddings[X[i][j].lower()]
            except:
                 embeddings_out[i][j]=np.zeros((50,))
    
    return embeddings_out
    
embeddings_matrix_train=embeddings_outputs(x_train)
embeddings_matrix_test=embeddings_outputs(x_test)


C:\Users\taran\AppData\Local\Temp/ipykernel_25508/2307853384.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i]=X[i].split()


## Building Model

In [85]:
from tensorflow.keras.utils import to_categorical

In [86]:
y_train=to_categorical(y_train,num_classes=5)
y_test=to_categorical(y_test,num_classes=5)

In [87]:
from keras.models import Sequential
from keras.layers import *

In [88]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 10, 64)            29440     
                                                                 
 dropout_8 (Dropout)         (None, 10, 64)            0         
                                                                 
 lstm_10 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 5)                 325       
                                                                 
 activation_6 (Activation)   (None, 5)                 0         
                                                                 
Total params: 62,789
Trainable params: 62,789
Non-trai

In [89]:
hist=model.fit(embeddings_matrix_train,y_train,epochs=94,batch_size=64,shuffle=True,validation_split=0.2)

Epoch 1/94
2/2 [==============================] - 3s 691ms/step - loss: 1.6269 - accuracy: 0.1524 - val_loss: 1.6108 - val_accuracy: 0.2222
Epoch 2/94
2/2 [==============================] - 0s 33ms/step - loss: 1.5773 - accuracy: 0.3619 - val_loss: 1.6175 - val_accuracy: 0.2222
Epoch 3/94
2/2 [==============================] - 0s 40ms/step - loss: 1.5368 - accuracy: 0.4000 - val_loss: 1.6292 - val_accuracy: 0.1852
Epoch 4/94
2/2 [==============================] - 0s 35ms/step - loss: 1.5203 - accuracy: 0.3429 - val_loss: 1.6462 - val_accuracy: 0.2593
Epoch 5/94
2/2 [==============================] - 0s 35ms/step - loss: 1.4870 - accuracy: 0.3619 - val_loss: 1.6622 - val_accuracy: 0.2593
Epoch 6/94
2/2 [==============================] - 0s 33ms/step - loss: 1.4931 - accuracy: 0.3524 - val_loss: 1.6624 - val_accuracy: 0.2963
Epoch 7/94
2/2 [==============================] - 0s 33ms/step - loss: 1.4421 - accuracy: 0.3619 - val_loss: 1.6437 - val_accuracy: 0.2593
Epoch 8/94
2/2 [==========

In [90]:
model.evaluate(embeddings_matrix_test,y_test)

2/2 [==============================] - 0s 4ms/step - loss: 2.2631 - accuracy: 0.5536


[2.2631256580352783, 0.5535714030265808]